# Impute Missing Data

In [1]:
#import Pkg; Pkg.add("Tables")
using LowRankModels, Random, LinearAlgebra, Plots, Statistics,DataFrames,LaTeXStrings, CSV,Tables
pyplot()

Plots.PyPlotBackend()

$\textbf{Read cleaned data file}$

In [2]:
# Read cleaned data files
raw_data = CSV.read("camry_clean.csv");
Real_data = raw_data[:,18:29]
header = names(Real_data)
head(Real_data)

,listed price,year,mileage,Accident,Past Owners,is_Fleet_Use,is_Personal_Use
,Int64⍰,Int64,Int64,Int64,Int64,Int64,Int64
1,19000,2019,38962,0,1,0,1
2,7494,2012,146450,0,2,0,1
3,7226,2012,117965,0,2,0,1
4,16499,2019,38661,0,1,1,0
5,12000,2014,53503,0,1,0,1
6,11881,2018,139784,0,1,0,1


In [3]:
# The abserve space will not contain the missing data
Real_data = convert(Matrix,Real_data)
Ω_obs = findall(iszero,ismissing.(Real_data[:,:]))
Ω_missing = findall(!iszero,ismissing.(Real_data[:,:]));
Ω_obs_price = findall(iszero,ismissing.(Real_data[:,1]));

$\textbf{Feed into the GLRM}$

In [8]:
Random.seed!(1)
nn = size(Ω_obs,1)

loss = Loss[HuberLoss(), HuberLoss(), HuberLoss(),BvSLoss(),BvSLoss(),WeightedHingeLoss(),LogisticLoss(),LogisticLoss(),LogisticLoss(),LogisticLoss(),LogisticLoss(),LogisticLoss()]
#reg = QuadReg(0.001)
#rx = ZeroReg()
reg = NonNegConstraint()
k = 8
glrm = GLRM(Real_data,loss,reg,reg,k,obs = Ω_obs)
add_offset!(glrm)

# fit low rank model
fit!(glrm,ProxGradParams(max_iter=600))
Impute_data = impute(glrm)
MAE = sum(abs.(vec((Impute_data - Real_data)[Ω_obs_price])))/size(Ω_obs_price,1)  # MAE of price
    

Fitting GLRM
Iteration 10: objective value = 3.2235218765603054e8
Iteration 20: objective value = 1.2189491505977555e8
Iteration 30: objective value = 1.0950887204662155e8
Iteration 40: objective value = 1.0727892591832066e8
Iteration 50: objective value = 1.0629020120479745e8
Iteration 60: objective value = 1.0587883722192697e8
Iteration 70: objective value = 1.0545001000636382e8
Iteration 80: objective value = 1.0512510198805815e8
Iteration 90: objective value = 1.0467366029266475e8
Iteration 100: objective value = 1.0413833714249192e8
Iteration 110: objective value = 1.032539109736074e8
Iteration 120: objective value = 9.628293891353008e7
Iteration 130: objective value = 7.26769977759358e7
Iteration 140: objective value = 3.407350553041377e7
Iteration 150: objective value = 8.52965382135115e6
Iteration 160: objective value = 3.6579473407912776e6
Iteration 170: objective value = 3.296098509159269e6
Iteration 180: objective value = 3.251871466777078e6


55.281111398721016

$\textbf{Compare the imputed data set with the real data set}$

In [9]:
Impute_data

8495×12 Array{Number,2}:
 18965.2   2077.36  39001.9        1  1  …  false  false  false  false  false
  7480.22  2072.9       1.4652e5   1  4     false   true  false  false  false
  7097.74  1764.68      1.17955e5  1  2     false  false  false  false  false
 16602.6   1860.12  38694.1        1  1     false  false  false  false  false
 11931.7   1585.22  53486.8        1  1     false  false  false  false  false
 11978.3   2415.02      1.39845e5  1  2  …  false  false  false  false  false
 12959.1   1548.31  40207.9        1  1     false  false  false  false  false
 20053.5   1947.59  16240.1        1  1     false  false  false  false  false
 18122.8   1776.48  16406.0        1  1     false  false  false  false  false
 16559.4   1791.24  32096.1        1  1     false  false  false  false  false
 16969.0   1636.9   12905.7        1  1  …  false  false  false  false  false
 12847.5   1491.32  35332.9        1  1     false  false  false  false  false
  7703.43  2071.62      1.44104e5  1  4

In [10]:
Real_data

8495×12 Array{Union{Missing, Int64},2}:
 19000  2019   38962  0  1  0  1  1  0  0  0  0
  7494  2012  146450  0  2  0  1  1  0  0  0  0
  7226  2012  117965  0  2  0  1  1  0  0  0  0
 16499  2019   38661  0  1  1  0  1  0  0  0  0
 12000  2014   53503  0  1  0  1  1  0  0  0  0
 11881  2018  139784  0  1  0  1  1  0  0  0  0
 13000  2018   40245  0  1  0  1  1  0  0  0  0
 20000  2018   16175  0  1  1  0  1  0  0  0  0
 18028  2018   16414  0  1  0  1  1  0  0  0  0
 16500  2018   32106  0  1  0  1  1  0  0  0  0
 16899  2019   12852  0  1  0  1  1  0  0  0  0
 12801  2014   35273  0  2  1  0  1  0  0  0  0
  7750  2012  144032  0  3  1  0  1  0  0  0  0
     ⋮                      ⋮              ⋮  
 33995  2020   12385  0  1  0  1  0  0  0  0  1
 29540  2018   34035  0  1  0  1  0  0  0  0  1
 26500  2019   96660  0  1  0  1  0  0  0  0  1
 18377  2017   52477  0  1  0  1  0  0  0  0  1
 17995  2015   65607  0  1  0  1  0  0  0  0  1
 18995  2017   73014  0  1  0  1  0  0  0  0  1
 

$\textbf{Output new CSV file}$

In [11]:
# only impute missing data
Impute_missing_data = impute_missing(glrm)

8495×12 Array{Number,2}:
 19000  2019   38962  0  1  0  1  1  0  0  0  0
  7494  2012  146450  0  2  0  1  1  0  0  0  0
  7226  2012  117965  0  2  0  1  1  0  0  0  0
 16499  2019   38661  0  1  1  0  1  0  0  0  0
 12000  2014   53503  0  1  0  1  1  0  0  0  0
 11881  2018  139784  0  1  0  1  1  0  0  0  0
 13000  2018   40245  0  1  0  1  1  0  0  0  0
 20000  2018   16175  0  1  1  0  1  0  0  0  0
 18028  2018   16414  0  1  0  1  1  0  0  0  0
 16500  2018   32106  0  1  0  1  1  0  0  0  0
 16899  2019   12852  0  1  0  1  1  0  0  0  0
 12801  2014   35273  0  2  1  0  1  0  0  0  0
  7750  2012  144032  0  3  1  0  1  0  0  0  0
     ⋮                      ⋮              ⋮  
 33995  2020   12385  0  1  0  1  0  0  0  0  1
 29540  2018   34035  0  1  0  1  0  0  0  0  1
 26500  2019   96660  0  1  0  1  0  0  0  0  1
 18377  2017   52477  0  1  0  1  0  0  0  0  1
 17995  2015   65607  0  1  0  1  0  0  0  0  1
 18995  2017   73014  0  1  0  1  0  0  0  0  1
 20995  2017   1

In [152]:
# OUTPUT CSV file
df = Tables.table(Impute_missing_data;header)
CSV.write("Camry_impute_missing_data.csv",df)

"Camry_impute_missing_data.csv"